Add three new features onto existing dataframe, which is Jaro distance,Normalized compression distance, and Smith-Waterman algorithm to compute similarity.

In [1]:
# Get data to work on. Join the table product_descriptions.csv and train.
import re
import string
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn.feature_extraction import stop_words

DESC_FILE = "product_descriptions.csv"
TRAIN_FILE = "train.csv"
MODIFIED_FILE = "modified_train.csv"

df_desc = pd.read_csv(DESC_FILE)
df_train = pd.read_csv(TRAIN_FILE,encoding="latin-1")
df_train_desc = df_train.set_index('product_uid').join(df_desc.set_index('product_uid'))
df_modified_train = pd.read_csv(MODIFIED_FILE)

FileNotFoundError: File b'product_descriptions.csv' does not exist

### Add clean description for computing score

In [ ]:
clean_desc = pd.read_csv("clean_desc.csv",header=None)

Try to add clean description to df_train_desc 

In [ ]:
df_train.head()
tryout = df_train.iloc[:20]
ap = clean_desc.iloc[:20]
merged = tryout.assign(clean_desc = ap)

In [ ]:
df_train_desc = df_train_desc.assign(clean_desc=clean_desc)

In [ ]:
# Extract the search_terms, description, titles for computing three metrics.
values = {'product_descption': "", 'product_title': "", 'search_term': "", 'relevance': ""}
df_train_desc = df_train_desc.fillna(value=values)

desc_col = df_train_desc['product_description']
search_col = df_train_desc["search_term"]
title_col = df_train_desc["product_title"]
clean_term_col = df_modified_train["cleaned_terms"]
terms_neighbors_col = df_modified_train["terms_neighbour"]

## To do: 
1. develop prototype of three metrics computation.
2. feed on clean description and original data.

## Compute Jaro distance on two pd.series that represents text columns.
https://rosettacode.org/wiki/Jaro_distance#Python

In [ ]:
def jaro(s, t):
    s_len = len(s)
    t_len = len(t)

    if s_len == 0 and t_len == 0:
        return 1

    match_distance = (max(s_len, t_len) // 2) - 1

    s_matches = [False] * s_len
    t_matches = [False] * t_len

    matches = 0
    transpositions = 0

    for i in range(s_len):
        start = max(0, i-match_distance)
        end = min(i+match_distance+1, t_len)

        for j in range(start, end):
            if t_matches[j]:
                continue
            if s[i] != t[j]:
                continue
            s_matches[i] = True
            t_matches[j] = True
            matches += 1
            break
    if matches == 0:
        return 0
    k = 0
    for i in range(s_len):
        if not s_matches[i]:
            continue
        while not t_matches[k]:
            k += 1
        if s[i] != t[k]:
            transpositions += 1
        k += 1
    return ((matches / s_len) +
            (matches / t_len) +
            ((matches - transpositions/2) / matches)) / 3

Idea: to do the computation of two documents' similarity score. I can think of using each search terms and find the average(? could be weighted for terms that appeared more frequently) of Jaro distiance with respect to each terms in descriptions.  
The using of Jaro score require select a threshold. Jaro describe the editing distance, for example soup and sour has score as 0.8333, but they are definite not the same thing. 0.8 as a threshold should be sufficient. In the context of product description, we can rule out such random similar words' appearance. 

In [ ]:
def getJaroScoreOnDocs(query, long_text):
    #transform query and long_text to list of words.
    query_ls = query.split()
    long_text_ls = long_text.split()
    
    total_J_score = 0
    for i in query_ls:
        j_score_in_i = sum([jaro(i,j) for j in long_text_ls if jaro(i,j)>0.83])
        total_J_score += j_score_in_i
    return total_J_score
#desc_col.iloc[0]
#terms_neighbors_col.iloc[0]

To get the total jaro distance w.r.t each search terms and description in a column.

In [ ]:
def createJaroCol(df,query_col_name,text_col_name,new_col_name):
    # Could combine title and description as a unit to compute Jaro score. It will be higher but as one score, easy to compute.
    # compute all jscore in a list
    j_score_ls = []
    for i in range(len(df)):
        query = df[query_col_name].iloc[i]
        long_text = df[text_col_name].iloc[i]
        j_score = getJaroScoreOnDocs(query,long_text)
        j_score_ls.append(j_score)
    df[new_col_name] = j_score_ls
    return None

In [ ]:
# this takes about 3mins to create a jaro score on query and description.
createJaroCol(df_train_desc,"search_term","product_description","jscore_query_desc")
createJaroCol(df_train_desc,"search_term","product_title","jscore_query_title")


In [ ]:
# Inspect the result in df_train_desc.df_train_desc.headad()
# Prepare for writing csv file for two new features.
output1 = df_train_desc.drop(["product_title","search_term","relevance","product_description"],axis=1)
# Make csv file.
output1.to_csv("two_jscore_feature.csv")

### Using Smith–Waterman algorithm to compute string similarity
sequence alignment is a way of arranging the sequences of DNA, RNA, or protein to identify regions of similarity that may be a consequence of functional.
The SW algorithm will yield a matrix represents all possible alignment's score. We can use two string to get the optimal alignment sequence and compute the alignment score. For all the combination between search terms and description, we can collect the overall optimal alignment sequence score. For example, screw and driver 
https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm#Substitution_matrix

In [ ]:

def smith_waterman(a: str, b: str, alignment_score: float = 1, gap_cost: float = 1) -> float:
    """
  Compute the Smith-Waterman alignment score for two strings.
  See https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm#Algorithm
  This implementation has a fixed gap cost (i.e. extending a gap is considered
  free). In the terminology of the Wikipedia description, W_k = {c, c, c, ...}.
  This implementation also has a fixed alignment score, awarded if the relevant
  characters are equal.
  Kinda slow, especially for large (50+ char) inputs.
    """
# H holds the alignment score at each point, computed incrementally
    H = np.zeros((len(a) + 1, len(b) + 1))
    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
      # The score for substituting the letter a[i-1] for b[j-1]. Generally low
      # for mismatch, high for match.
            match = H[i-1,j-1] + (alignment_score if a[i-1] == b[j-1] else 0)
      # The scores for for introducing extra letters in one of the strings (or
      # by symmetry, deleting them from the other).
            delete = H[1:i,j].max() - gap_cost if i > 1 else 0
            insert = H[i,1:j].max() - gap_cost if j > 1 else 0
            H[i,j] = max(match, delete, insert, 0)
  # The highest score is the best local alignment.
  # For our purposes, we don't actually care _what_ the alignment was, just how
  # aligned the two strings were.
    return H.max()

In [ ]:
def getSWscore(query, long_text):
    """
    param: query is the search query as a string.
    param: text is the long text to compute the similarity.
    return the number of significant alignment strings in both text. ie. the number of similar terms in query and long_text.
    """
    query_ls = query.split()
    long_text_ls = long_text.split()
    
    sw_score = []
    for i in query_ls:
        score = sum([smith_waterman(i,j)for j in long_text_ls if smith_waterman(i,j)>=4.0 ])
        #print([smith_waterman(i,j) if smith_waterman(i,j)>=4.0 else 0 for j in long_text_ls])
        sw_score.append(score)
    return round(sum(sw_score)/5)

In [ ]:
def createSWscoreCol(df,query_col_name,long_text_col_name,new_col_name):
    first_col = df[query_col_name]
    second_col = df[long_text_col_name]
    
    score_ls = []
    for i in range(len(first_col)):
        score_ls.append(getSWscore(first_col.iloc[i],second_col.iloc[i]))
    df[new_col_name] = score_ls
    return df

In [ ]:
# Get the SW score for search and title.
newdf = createSWscoreCol(df_train_desc,"search_term","product_title","search_title_SW")

In [ ]:
# Prepare to write out the three new features.
three_feature = newdf.drop(["product_title","search_term","relevance","product_description"],axis=1)
three_feature.to_csv("three_features.csv")

In [ ]:
# Get the SW score for search and description.
newdf2 = createSWscoreCol(df_train_desc,"search_term","product_description","search_desc_SW")

In [ ]:
# Prepare to write out the four new features.
newdf2.drop(["product_title","search_term","relevance","product_description"],axis=1).to_csv("four_features.csv")
newdf2.to_csv("four_features.csv")

### Using Normalized compression distance.

The normalized compression distance is a way to describe the similarity between any files by comparing the bit sizes. The NCD is a non-negative number 0 ≤ r ≤ 1 representing how different the two files are. Smaller numbers represent more similar files. 

In [ ]:
import lzma
import sys

def computeNCD(string1,string2):
    """
    params: string1 is the query term
    params: string2 is the word in long-text, like title, description.
    """
    # Get concated strings and transform to bytes-like object for lzma.compress.
    concat_str = string1+string2
    string1 = bytes(string1,'utf-8')
    string2 = bytes(string2,'utf-8')
    concat_str = bytes(concat_str,'utf-8')
    
    # Get the compressed file for each string.
    str1_comp = lzma.compress(string1)  # compress file 1
    str2_comp = lzma.compress(string2)  # compress file 2
    concat_str_comp = lzma.compress(concat_str)  # compress file concatenated

    # print len() of each file
    # print(len(str1_comp), len(str2_comp), len(concat_str_comp), sep=' ', end='\n')

    # magic happens here
    ncd = (len(concat_str_comp) - min(len(str1_comp), len(str2_comp))) / \
    max(len(str1_comp), len(str2_comp))

    # print(ncd)
    return ncd

In [ ]:
# A test for computeNCD function
str1 = df_train_desc.iloc[0][1] # the title
str2 = df_train_desc.iloc[0][2] # the search
str3 = df_train_desc.iloc[0][4] # the desc

#### The usage of Normalized Compression Distance:</t>
Since we only compare the bits difference from one string to another, so we need to average the score between word of query and word in title or description. So we need to find the overall similarity between the two words. To do so, I will compute each combination between word in query and word in long-text, and take the average as a final similarity score.

In [ ]:
def createNCDCol(df,search_name, long_text_name,new_col_name):
    NCD_score_ls = []
    for i in range(len(df)):
        str1 = df[search_name].iloc[i]
        str2 = df[long_text_name].iloc[i]
        NCD_score_ls.append(np.mean([computeNCD(a,b) for a in str1.split() for b in str2.split()]))
    df[new_col_name] = NCD_score_ls
    return df

In [ ]:
# try on first 100 lines of df_train_desc
tryout = createNCDCol(df_train_desc,"search_term","product_title","NCD_query_title")